# Tarea Big Data
## Rafael Farias Poblete

In [1]:
import findspark
from pyspark import SparkContext, SparkConf, SQLContext
import csv
import geopandas as gpd
from pyspark.sql.functions import substring, expr, countDistinct
from shapely import wkt
findspark.init()  # Con este no me tira error de JVM.

In [2]:
# Se setea el Master y se le da nombre a la aplicación.
conf = SparkConf().setMaster("local").setAppName("Tarea Analisis de BigData")

# Se inicia el cluster Spark.
sc = SparkContext.getOrCreate(conf=conf)

# Se inicia SQLContext desde el cluster de Spark.
sqlContext = SQLContext(sc)

In [3]:
FilePath = '/home/rafa/Dropbox/Linux_MDS/BDAnalytics/sprint1/data'

FileName1 = 'wifi_2017.csv'
FileName2 = 'wifi_2018.csv'
FileName3 = 'wifi_2019.csv'

FullPath1 = FilePath + '/' + FileName1
FullPath2 = FilePath + '/' + FileName2
FullPath3 = FilePath + '/' + FileName3

df_2017 = sqlContext.read.csv(FullPath1, header=True)
df_2018 = sqlContext.read.csv(FullPath2, header=True)
df_2019 = sqlContext.read.csv(FullPath3, header=True)

df_2017 = df_2017.withColumnRenamed('data_source', 'data')

df_2017 = df_2017.drop('range', 'created')

# Ahora ordeno las columnas para que todas tengan el mismo orden.
df_2017 = df_2017.select('id', 'bssid', 'lat', 'lon', 'updated', 'data')

df_unidos = ((df_2017.union(df_2018)).union(df_2019)).distinct()
print('El dataframe que contiene todos los csv es el siguiente:\n')
df_unidos.show(truncate=False)
df_unidos = df_unidos.drop('updated', 'data')

El dataframe que contiene todos los csv es el siguiente:

+----+------------+-----------+-----------+----------+----+
|id  |bssid       |lat        |lon        |updated   |data|
+----+------------+-----------+-----------+----------+----+
|802 |00040E94ED73|50.08081200|8.28433922 |1428513555|0   |
|1076|00095BDC1C2A|53.68903505|9.99661764 |1428513555|0   |
|1198|000B6B2F0FAC|49.68090465|8.62257210 |1428513555|0   |
|1304|000B850DA2B1|49.39064487|8.63899747 |1428513555|0   |
|1371|00040EE37419|49.66931085|9.00361495 |1428513555|0   |
|1484|00040ED8D824|51.16131842|13.47315431|1428513555|0   |
|1893|0001E30E688B|49.54164425|8.63952318 |1428513555|0   |
|2077|00040E8F92CD|50.93837781|6.94276197 |1428513555|0   |
|2586|000CF67F25B8|49.49755490|8.55307102 |1428513555|0   |
|2785|000FBB11D5A8|48.37995887|10.92875719|1428513555|0   |
|2971|000FB5B82AFC|50.07127597|8.43380130 |1428513555|0   |
|3085|000352A70940|50.10473795|8.66833063 |1428513555|0   |
|3202|00040EEDE668|53.56497360|9.80239220 

In [4]:
# Ahora creo el df solo con la RM según coordenadas de google maps.
f1_fabricante = df_unidos.filter((df_unidos.lat >= -33.65) &
                           (df_unidos.lat <= -33.28) &
                           (df_unidos.lon >= -70.81) &
                           (df_unidos.lon <= -70.50))

# Credo el df final de Stgo.
# Separo la columna bssid en una que contendrá Media_mac y otra que contendrá
# Id_fabricante.
f1_fabricante = f1_fabricante.withColumn('Id_fabricante', expr('substring(bssid,1,length(bssid)-6)')).withColumn(
    'Media_mac', expr('substring(bssid,7,length(bssid)-6)')).drop('bssid')
print('El dataframe de Santiago es el siguiente:\n')
f1_fabricante.show()

El dataframe de Santiago es el siguiente:

+-------+------------+------------+-------------+---------+
|     id|         lat|         lon|Id_fabricante|Media_mac|
+-------+------------+------------+-------------+---------+
|8849791|-33.39275272|-70.57426126|       1CE6C7|   F010E6|
|8849792|-33.39250351|-70.57437245|       1CE6C7|   F010E5|
|8850002|-33.39918647|-70.58116012|       001EE5|   3444A9|
|8850139|-33.39342478|-70.57411691|       64E950|   33D911|
|8850625|-33.40811106|-70.58709986|       FC94E3|   0C3C64|
|8851402|-33.39346036|-70.57409400|       34DBFD|   BFF298|
|8991482|-33.40576570|-70.56381641|       001DCF|   203FD1|
|8991542|-33.40188118|-70.56105160|       2CAB25|   85EF3F|
|8991749|-33.39870261|-70.55496093|       004F62|   1D5873|
|8992564|-33.41227703|-70.57368460|       40CBA8|   CCBD9D|
|8992611|-33.40116664|-70.56038709|       B4750E|   38BC6E|
|8993378|-33.42087583|-70.56632524|       14ABF0|   DD5A70|
|8994248|-33.41674528|-70.57156504|       70105C|   7CD8F

In [5]:
# Ahora trabajo con el archivo de texto.
dict_vendor_id = dict()

for lig in open('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/sprint1/data/'
                'oui.txt'):
    if 'base 16' in lig:
        num, sep, txt = lig.strip().partition('(base 16)')
        dict_vendor_id[num.strip()] = txt.strip()

# Transformo el diccionario en csv para mejor manipulación.
with open('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/sprint1/oui.csv',
          'w') as f:
    w = csv.writer(f)
    w.writerows(dict_vendor_id.items())

# Creo el df
df_oui = sqlContext.read.csv('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/'
                             'sprint1/oui.csv',
                             header=False)
print('El dataframe del archivo OUI.txt es el siguiente:\n')
df_oui.show(truncate=False)


El dataframe del archivo OUI.txt es el siguiente:

+------+------------------------------------------------------------+
|_c0   |_c1                                                         |
+------+------------------------------------------------------------+
|002272|American Micro-Fuel Device Corp.                            |
|00D0EF|IGT                                                         |
|086195|Rockwell Automation                                         |
|F4BD9E|Cisco Systems, Inc                                          |
|5885E9|Realme Chongqing MobileTelecommunications Corp Ltd          |
|BC2392|BYD Precision Manufacture Company Ltd.                      |
|405582|Nokia                                                       |
|A4E31B|Nokia                                                       |
|D89790|Commonwealth Scientific and Industrial Research Organisation|
|883A30|Aruba, a Hewlett Packard Enterprise Company                 |
|B8A58D|Axe Group Holdings Limited     

In [6]:
# Ahora me preocupo de revisar los archivos de geolocalización.
# En el Archivo Manzana_Precensal.shp se encuentra toda la info solicitada
# salvo por la ciudad.
Manzana_Precensal = gpd.read_file('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/'
                                  'sprint1/data/'
                                  'Manzana_Precensal.shp')

# Elimino columnas innecesarias.
Manzana_Precensal = Manzana_Precensal.drop(['DES_REGI', 'MANZENT', 'COMUNA',
                                            'PROVINCIA', 'DES_PROV', 'REGION',
                                            'COD_DIS'], axis=1)

print('El shape que contiene los datos solicitados es Manzana Precensal:\n')
print(Manzana_Precensal)

El shape que contiene los datos solicitados es Manzana Precensal:

       COD_ZON  COD_ENT   DES_COMU  \
0            3        1  CERRILLOS   
1            3        4  CERRILLOS   
2            3        5  CERRILLOS   
3            3        8  CERRILLOS   
4            3        9  CERRILLOS   
...        ...      ...        ...   
53048        1      504   PEÑAFLOR   
53049        1      503   PEÑAFLOR   
53050        1      502   PEÑAFLOR   
53051        1      505   PEÑAFLOR   
53052        1      501   PEÑAFLOR   

                                                geometry  
0      POLYGON ((-70.72166 -33.47385, -70.72197 -33.4...  
1      POLYGON ((-70.72136 -33.47420, -70.72160 -33.4...  
2      POLYGON ((-70.72105 -33.47455, -70.72124 -33.4...  
3      POLYGON ((-70.72079 -33.47509, -70.72089 -33.4...  
4      POLYGON ((-70.72052 -33.47563, -70.72052 -33.4...  
...                                                  ...  
53048  POLYGON ((-70.87621 -33.61954, -70.87751 -33.6...  
5304

In [7]:
# Construcción de futures.
# Fabricante.
# Uno los df_stgo y df_oui a través de un join y además le solicito que lo haga
# en donde Id_fabricante sea idéntico a _c0 del df_oui.
f1_fabricante = f1_fabricante.join(df_oui).where(f1_fabricante["Id_fabricante"] == df_oui["_c0"])
f1_fabricante = f1_fabricante.drop('_c0')
f1_fabricante = f1_fabricante.withColumnRenamed('_c1', 'Fabricante')
print('El dataframe con el primer future es el siguiente:\n')
f1_fabricante.show(truncate=False)

El dataframe con el primer future es el siguiente:

+-------+------------+------------+-------------+---------+--------------------------------------------+
|id     |lat         |lon         |Id_fabricante|Media_mac|Fabricante                                  |
+-------+------------+------------+-------------+---------+--------------------------------------------+
|8849791|-33.39275272|-70.57426126|1CE6C7       |F010E6   |Cisco Systems, Inc                          |
|8849792|-33.39250351|-70.57437245|1CE6C7       |F010E5   |Cisco Systems, Inc                          |
|8850002|-33.39918647|-70.58116012|001EE5       |3444A9   |Cisco-Linksys, LLC                          |
|8850139|-33.39342478|-70.57411691|64E950       |33D911   |Cisco Systems, Inc                          |
|8850625|-33.40811106|-70.58709986|FC94E3       |0C3C64   |Technicolor CH USA Inc.                     |
|8851402|-33.39346036|-70.57409400|34DBFD       |BFF298   |Cisco Systems, Inc                          |
|89

In [8]:
# Información geográfica (ciudad, comuna, zona censal, manzana)
# Transformo df_stgo a pandas para poder trabajarlo con geopandas
df_stgo_pandas = f1_fabricante.toPandas()

# Ahora creo un geopandas para indicarle los points de lat y lon.
df_stgo_geop = gpd.GeoDataFrame(df_stgo_pandas,
                                geometry=gpd.points_from_xy
                                (df_stgo_pandas.lon, df_stgo_pandas.lat))

df_stgo_geop = df_stgo_geop.drop(columns=['lat', 'lon'])

# Le indico el CRS para que quede igual a Manzana_Precensal.
df_stgo_geop.crs = 'EPSG:4674'

# Ahora hago un join con geopandas entre df_stgo_geop y Manzana_Precensal.
join_stgo_manzana = gpd.sjoin(df_stgo_geop, Manzana_Precensal, op='within',
                              how='inner')

join_stgo_manzana = join_stgo_manzana.drop(columns=['index_right'])
join_stgo_manzana['str_geom'] = join_stgo_manzana.geometry.apply(lambda x: wkt.
                                                                 dumps(x))
join_stgo_manzana = (join_stgo_manzana.drop(columns=['geometry'])
                     ).rename(columns={'str_geom': 'geometry',
                                       'COD_ZON': 'Zona_Censal',
                                       'COD_ENT': 'Manzana_Censal',
                                       'DES_COMU': 'Comuna'})

# Ahora lo vuelvo a pasar a pyspark.
f1_georeferencia = sqlContext.createDataFrame(join_stgo_manzana)

print('Dataset con el nuevo future de información geográfica agregado\n')
f1_georeferencia.show(truncate=False)

Dataset con el nuevo future de información geográfica agregado

+--------+-------------+---------+-----------------------------+-----------+--------------+--------+-------------------------------------------------+
|id      |Id_fabricante|Media_mac|Fabricante                   |Zona_Censal|Manzana_Censal|Comuna  |geometry                                         |
+--------+-------------+---------+-----------------------------+-----------+--------------+--------+-------------------------------------------------+
|8849791 |1CE6C7       |F010E6   |Cisco Systems, Inc           |5          |33            |VITACURA|POINT (-70.5742612600000001 -33.3927527199999972)|
|8849792 |1CE6C7       |F010E5   |Cisco Systems, Inc           |5          |33            |VITACURA|POINT (-70.5743724499999985 -33.3925035099999974)|
|8850139 |64E950       |33D911   |Cisco Systems, Inc           |5          |33            |VITACURA|POINT (-70.5741169100000008 -33.3934247799999966)|
|8851402 |34DBFD       |BFF298

In [9]:
# Cuento las ocurrencias de cada fabricante.
f1_georeferencia.groupBy('Fabricante').count().orderBy('count', 
                                                            ascending=False)\
                                                      .show(truncate=False)

# Con la info de arriba tomo la decisión de agregar los 3 primeros fabricantes
# como futures.
# Paso a pandas para aplicar apply y lambda y poner un 1 donde encuentra el
# nombre del fabricante y 0 en otros casos.
f1_georeferencia = f1_georeferencia.toPandas()

f1_georeferencia['q_ARRIS_Group'] = f1_georeferencia.apply\
    (lambda x: 1 if (x["Fabricante"]) == 'ARRIS Group, Inc.' else 0, axis=1)

f1_georeferencia['q_Cisco_Systems_Inc'] = f1_georeferencia.apply\
    (lambda x: 1 if (x["Fabricante"]) == 'Cisco Systems, Inc' else 0, axis=1)

f1_georeferencia['q_Technicolor'] = f1_georeferencia.apply\
    (lambda x: 1 if (x["Fabricante"]) == 'Technicolor CH USA Inc.' else 0,
     axis=1)

suma = f1_georeferencia['q_ARRIS_Group'].sum() +\
    f1_georeferencia['q_Cisco_Systems_Inc'].sum() +\
    f1_georeferencia['q_Technicolor'].sum()

+--------------------------------------------+-----+
|Fabricante                                  |count|
+--------------------------------------------+-----+
|ARRIS Group, Inc.                           |2884 |
|Cisco Systems, Inc                          |2134 |
|Technicolor CH USA Inc.                     |1440 |
|Cisco-Linksys, LLC                          |1219 |
|TP-LINK TECHNOLOGIES CO.,LTD.               |1096 |
|PEGATRON CORPORATION                        |1039 |
|Technicolor Delivery Technologies Belgium NV|873  |
|D-Link International                        |660  |
|Gemtek Technology Co., Ltd.                 |610  |
|Thomson Telecom Belgium                     |510  |
|ASUSTek COMPUTER INC.                       |455  |
|Apple, Inc.                                 |451  |
|SHENZHEN GONGJIN ELECTRONICS CO.,LT         |449  |
|D-Link Corporation                          |384  |
|Ruckus Wireless                             |331  |
|Belkin International Inc.                   |

In [10]:
# Generar lo otros futures que corresponden a la proporción.     
f1_georeferencia['p_ARRIS_Group'] = f1_georeferencia.apply\
    (lambda x: 1/suma if (x["Fabricante"]) == 'ARRIS Group, Inc.' else 0, axis=1)

f1_georeferencia['p_Cisco_Systems_Inc'] = f1_georeferencia.apply\
    (lambda x: 1/suma if (x["Fabricante"]) == 'Cisco Systems, Inc' else 0, axis=1)

f1_georeferencia['p_Technicolor'] = f1_georeferencia.apply\
    (lambda x: 1/suma if (x["Fabricante"]) == 'Technicolor CH USA Inc.' else 0,
     axis=1)

proporcion = f1_georeferencia['p_ARRIS_Group'].sum() +\
    f1_georeferencia['p_Cisco_Systems_Inc'].sum() +\
    f1_georeferencia['p_Technicolor'].sum()   

f2_sum_prop = sqlContext.createDataFrame(f1_georeferencia)
f2_sum_prop.show()

+--------+-------------+---------+--------------------+-----------+--------------+--------+--------------------+-------------+-------------------+-------------+--------------------+--------------------+-------------+
|      id|Id_fabricante|Media_mac|          Fabricante|Zona_Censal|Manzana_Censal|  Comuna|            geometry|q_ARRIS_Group|q_Cisco_Systems_Inc|q_Technicolor|       p_ARRIS_Group| p_Cisco_Systems_Inc|p_Technicolor|
+--------+-------------+---------+--------------------+-----------+--------------+--------+--------------------+-------------+-------------------+-------------+--------------------+--------------------+-------------+
| 8849791|       1CE6C7|   F010E6|  Cisco Systems, Inc|          5|            33|VITACURA|POINT (-70.574261...|            0|                  1|            0|                 0.0|1.548467017652524E-4|          0.0|
| 8849792|       1CE6C7|   F010E5|  Cisco Systems, Inc|          5|            33|VITACURA|POINT (-70.574372...|            0|      